# Python learning to rank (LTR) toolkit についての調査

ランク学習の１実装事例である「Python learning to rank (LTR) toolkit」について調査しました。

こちらを参考にいたしました。

https://github.com/jma127/pyltr

https://github.com/jma127/pyltr/blob/master/README.rst

## (1) 概要

Python 言語により実装されております。

実装されているのは「LambdaMART」と呼ばれる、決定木アルゴリズムの一種を使用したランク学習モデル、とのことです。


- LambdaMARTアルゴリズムについての解説は以下URLに記載されています
 
　　https://wellecks.wordpress.com/tag/lambdamart/

　　https://www.microsoft.com/en-us/research/publication/from-ranknet-to-lambdarank-to-lambdamart-an-overview/
 
  
- MART（勾配ブースティング決定木）についての参考文献は以下のURLにあります
  
　　http://smrmkt.hatenablog.jp/entry/2015/04/28/210039 
 
 
- 「LambdaMART」は、手法としては pairwise/listwise に分類されるようです。

　　こちら (https://en.wikipedia.org/wiki/Learning_to_rank#Pointwise_approach) の「List of methods」をご参照
 

- 入力データ（教師データ）形式は<a href="01-SVMRank-outline.ipynb"><b>SVMRank</b></a>と同じものが使用できます。

### (1-1) 教師データ

クエリーごとに、素性（featureベクトルと等価）と、そのランクを教師データとして用意します。

ランクの値が大きいほど、そのクエリーIDにおけるランキングが高い素性である、という意味になるようです。

教師データのフォーマット：(fit関数の引数。feature=10件とします)

| ランク | クエリーID | feature1 |  feature2 | ・・・ |  feature10 | 
| :---: | :---: | :---: |  :---: | :---: |  :---: | 
| 3 | qid:1 | 1:0.5 |  2:0.0 | ・・・ |  10:0.5 | 
| 2 | qid:1 | 1:0.2 |  2:1.0 | ・・・ |  10:0.1 | 
| 1 | qid:1 | 1:0.0 |  2:1.0 | ・・・ |  10:0.0 | 
| 3 | qid:2 | 1:0.0 |  2:0.1 | ・・・ |  10:0.2 | 
| 2 | qid:2 | 1:1.0 |  2:0.3 | ・・・ |  10:0.2 | 
| 1 | qid:2 | 1:1.0 |  2:0.5 | ・・・ |  10:1.0 | 


### (1-2) 学習

教師データを学習処理（LambdaMART）に渡すと、fit関数により学習を行い、モデルが生成されます。

### (1-3) 予測


テストデータをpredict関数に渡すと、予測結果が生成され、テストデータについてのランキング・スコアが得られます。


テストデータのフォーマット：(predict関数の引数＝前述の教師データのフォーマットと同一)

| ランク | クエリーID | feature1 |  feature2 | ・・・ |  feature10 | 
| :---: | :---: | :---: |  :---: | :---: |  :---: | 
| 1 | qid:9 | 1:0.0 |  2:0.0 | ・・・ |  10:0.0 | 
| 1 | qid:9 | 1:0.1 |  2:1.0 | ・・・ |  10:0.1 | 
| 1 | qid:9 | 1:0.5 |  2:0.0 | ・・・ |  10:0.3 | 


予測結果データのフォーマット：(predict関数の戻り＝テストデータと並びが同じになるようです)

| ランキング・スコア |
| :---: |
| -0.5 |
| 0.5 |
| 1.5 |

ただしこのランキング・スコアは、純粋にテストデータとランキング結果の対応だけに使用される想定であり、その値自体が何らかの指標となるものではないようです。

（すなわち、テストデータをランキング・スコアの降順に並べ替えて、ランキングを得る・・・といった利用を想定している様子）

したがって、テストデータが１件しかないばあい、この手法の予測結果は意味を持たないかと存じます。

## (2) 環境準備

### (2-1) GitHub からファイルを取得

```
MacBookPro-makmorit-jp:GitHub makmorit$ git clone https://github.com/jma127/pyltr.git
Cloning into 'pyltr'...
remote: Counting objects: 281, done.
remote: Total 281 (delta 0), reused 0 (delta 0), pack-reused 281
Receiving objects: 100% (281/281), 50.50 KiB | 0 bytes/s, done.
Resolving deltas: 100% (164/164), done.
MacBookPro-makmorit-jp:GitHub makmorit$ ls -al pyltr
total 56
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 ..
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .git
-rw-r--r--   1 makmorit  staff   259 May  3 13:55 .gitignore
-rw-r--r--   1 makmorit  staff   182 May  3 13:55 .travis.yml
-rw-r--r--   1 makmorit  staff  1479 May  3 13:55 LICENSE.txt
-rw-r--r--   1 makmorit  staff  2704 May  3 13:55 README.rst
-rw-r--r--   1 makmorit  staff   211 May  3 13:55 TODO.txt
drwxr-xr-x  11 makmorit  staff   374 May  3 13:55 docs
drwxr-xr-x   7 makmorit  staff   238 May  3 13:55 pyltr
-rwxr-xr-x   1 makmorit  staff    65 May  3 13:55 run_tests.sh
-rw-r--r--   1 makmorit  staff   494 May  3 13:55 setup.py
MacBookPro-makmorit-jp:GitHub makmorit$
```

### (2-2) ユニットテスト実行（環境のベリファイ）

run_tests.sh を実行させます。

事前に、内部でimportされる overrides モジュールを追加導入しています。

```
MacBookPro-makmorit-jp:GitHub makmorit$ cd pyltr
MacBookPro-makmorit-jp:pyltr makmorit$ pwd
/Users/makmorit/GitHub/pyltr
MacBookPro-makmorit-jp:pyltr makmorit$ ls -al
total 56
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 ..
drwxr-xr-x  12 makmorit  staff   408 May  3 13:55 .git
-rw-r--r--   1 makmorit  staff   259 May  3 13:55 .gitignore
-rw-r--r--   1 makmorit  staff   182 May  3 13:55 .travis.yml
-rw-r--r--   1 makmorit  staff  1479 May  3 13:55 LICENSE.txt
-rw-r--r--   1 makmorit  staff  2704 May  3 13:55 README.rst
-rw-r--r--   1 makmorit  staff   211 May  3 13:55 TODO.txt
drwxr-xr-x  11 makmorit  staff   374 May  3 13:55 docs
drwxr-xr-x   7 makmorit  staff   238 May  3 13:55 pyltr
-rwxr-xr-x   1 makmorit  staff    65 May  3 13:55 run_tests.sh
-rw-r--r--   1 makmorit  staff   494 May  3 13:55 setup.py
MacBookPro-makmorit-jp:pyltr makmorit$ 
MacBookPro-makmorit-jp:pyltr makmorit$ pip3 install overrides
Collecting overrides
  Downloading overrides-1.7.tar.gz
Building wheels for collected packages: overrides
  Running setup.py bdist_wheel for overrides ... done
  Stored in directory: /Users/makmorit/Library/Caches/pip/wheels/93/7f/68/cb4e994316c6b40d4ccf468475267fac7105459ee7c51fef9f
Successfully built overrides
Installing collected packages: overrides
Successfully installed overrides-1.7
MacBookPro-makmorit-jp:pyltr makmorit$ ./run_tests.sh
pyltr.metrics.tests.test_ap.TestAP.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_ap.TestAP.test_evaluate ... ok
pyltr.metrics.tests.test_dcg.TestDCG.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_dcg.TestDCG.test_evaluate ... ok
pyltr.metrics.tests.test_dcg.TestNDCG.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_dcg.TestNDCG.test_evaluate ... ok
pyltr.metrics.tests.test_err.TestERR.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_err.TestERR.test_evaluate ... ok
pyltr.metrics.tests.test_kendall.TestKendallTau.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_kendall.TestKendallTau.test_evaluate ... ok
pyltr.metrics.tests.test_roc.TestAUCROC.test_calc_swap_deltas ... ok
pyltr.metrics.tests.test_roc.TestAUCROC.test_evaluate ... ok

----------------------------------------------------------------------
Ran 12 tests in 0.532s

OK
MacBookPro-makmorit-jp:pyltr makmorit$ 
```

### (2-3) テストデータの準備

<a href="01-SVMRank-outline.ipynb"><b>SVMRank</b></a> の調査で使用したファイルの形式が利用できるようです。

適宜編集し、任意のフォルダーに配置します。

```
MacBookPro-makmorit-jp:pyltr makmorit$ ls -al data
total 40
drwxr-xr-x   6 makmorit  staff   204 May  3 14:00 .
drwxr-xr-x  14 makmorit  staff   476 May  3 14:00 ..
-rw-r--r--@  1 makmorit  staff  6148 May  3 14:01 .DS_Store
-rwxr-----@  1 makmorit  staff   124 Jul  3  2002 test.txt
-rwxr-----@  1 makmorit  staff   402 May  3 14:12 train.txt
-rwxr-----@  1 makmorit  staff   124 Apr 30 14:41 vali.txt
```

## (3) example を使用した実行例

以下のサイトの記述を参考にしています。

https://github.com/jma127/pyltr/blob/master/README.rst#example

### (3-0) データの読み込み

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../../../pyltr") #<--- ~/GitHub/pyltr
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

In [2]:
print(learning_dir)

/Users/makmorit/GitHub/pyltr


In [3]:
import pyltr

In [4]:
with open('data/train.txt') as trainfile, \
    open('data/vali.txt') as valifile, \
    open('data/test.txt') as evalfile:
    TX, Ty, Tqids, _ = pyltr.data.letor.read_dataset(trainfile)
    VX, Vy, Vqids, _ = pyltr.data.letor.read_dataset(valifile)
    EX, Ey, Eqids, _ = pyltr.data.letor.read_dataset(evalfile)

#### データのダンプ

In [5]:
print('Data for training')
for index, _ in enumerate(TX):
    print('qid=%s features=%s rank=%d' % (Tqids[index], str(TX[index]), Ty[index]))

Data for training
qid=1 features=[ 1.   1.   0.   0.2  0. ] rank=3
qid=1 features=[ 0.   0.   1.   0.1  1. ] rank=2
qid=1 features=[ 0.   1.   0.   0.4  0. ] rank=1
qid=1 features=[ 0.   0.   1.   0.3  0. ] rank=1
qid=2 features=[ 0.   0.   1.   0.2  0. ] rank=1
qid=2 features=[ 1.   0.   1.   0.4  0. ] rank=2
qid=2 features=[ 0.   0.   1.   0.1  0. ] rank=1
qid=2 features=[ 0.   0.   1.   0.2  0. ] rank=1
qid=3 features=[ 0.   0.   1.   0.1  1. ] rank=2
qid=3 features=[ 1.   1.   0.   0.3  0. ] rank=3
qid=3 features=[ 1.   0.   0.   0.4  1. ] rank=4
qid=3 features=[ 0.   1.   1.   0.5  0. ] rank=1


In [6]:
print('Data for validation')
for index, _ in enumerate(VX):
    print('qid=%s features=%s rank=%d' % (Vqids[index], str(VX[index]), Vy[index]))

Data for validation
qid=4 features=[ 1.   0.   0.   0.2  1. ] rank=4
qid=4 features=[ 1.   1.   0.   0.3  0. ] rank=3
qid=4 features=[ 0.   0.   0.   0.2  1. ] rank=2
qid=4 features=[ 0.   0.   1.   0.2  0. ] rank=1


In [7]:
print('Data for test')
for index, _ in enumerate(EX):
    print('qid=%s features=%s rank=%d' % (Eqids[index], str(EX[index]), Ey[index]))

Data for test
qid=5 features=[ 1.   0.   0.   0.2  1. ] rank=3
qid=5 features=[ 1.   1.   0.   0.3  0. ] rank=2
qid=5 features=[ 0.   0.   0.   0.2  1. ] rank=2
qid=5 features=[ 0.   0.   1.   0.2  0. ] rank=1


### (3-1) 学習処理

学習セットファイル data/train.txt を引数として学習処理を行うと、model オブジェクトにモデルが生成されます。

In [8]:
metric = pyltr.metrics.NDCG(k=3)
model = pyltr.models.LambdaMART(
    metric=metric,
    n_estimators=1000,
    learning_rate=0.02,
    max_features=1,
    query_subsample=0.5,
    max_leaf_nodes=10,
    min_samples_leaf=64,
    verbose=1,
)
model

In [9]:
monitor = pyltr.models.monitors.ValidationMonitor(VX, Vy, Vqids, metric=metric, stop_after=250)
model.fit(TX, Ty, Tqids, monitor=monitor)

 Iter  Train score  OOB Improve    Remaining                           Monitor Output 
    1       0.3056       0.0000        1.37s      C:      0.3056 B:      0.3056 S:  0
    2       0.3056       0.0000        1.28s      C:      0.3056 B:      0.3056 S:  1
    3       0.3333       0.0000        1.08s      C:      0.3056 B:      0.3056 S:  2
    4       0.3056       0.0000        1.01s      C:      0.3056 B:      0.3056 S:  3
    5       0.3333       0.0000        0.98s      C:      0.3056 B:      0.3056 S:  4
    6       0.3333       0.0000        0.98s      C:      0.3056 B:      0.3056 S:  5
    7       0.3056       0.0000        0.98s      C:      0.3056 B:      0.3056 S:  6
    8       0.3056       0.0000        0.95s      C:      0.3056 B:      0.3056 S:  7
    9       0.5158       0.0000        0.92s      C:      0.3056 B:      0.3056 S:  8
   10       0.5158       0.0000        0.90s      C:      0.3056 B:      0.3056 S:  9
   15       0.5158       0.0000        0.78s      C: 

### (3-2) 予測処理

テストデータファイル 'data/test.txt' を引数として予測処理処理を行うと、Epred に予測結果が格納されます。

ただし、得られたスコアは全て同値でした。

In [10]:
Epred = model.predict(EX)
Epred

array([ 0.,  0.,  0.,  0.])

### (3-3) 予測結果を参照

Epred に、ランキング・スコアが出力されます。

（ちなみに、テストデータのレコードと同じ並びになっているようです）

これをユーザープログラムなどで降順に整列し、ランキング結果として利用する想定のようです。

In [11]:
'''
    ランキングスコアで降順ソート
'''
ranking_array = []
for index, _ in enumerate(EX):
    ranking_array.append((Eqids[index], EX[index], Epred[index]))

sorted_array = sorted(ranking_array, key=lambda x:x[2], reverse=True)

In [12]:
'''
    ランキングを表示します
'''
for qid, features, score in sorted_array:
    print('qid=%s features=%s ranking score=%0.3f' % (qid, features, score))

qid=5 features=[ 1.   0.   0.   0.2  1. ] ranking score=0.000
qid=5 features=[ 1.   1.   0.   0.3  0. ] ranking score=0.000
qid=5 features=[ 0.   0.   0.   0.2  1. ] ranking score=0.000
qid=5 features=[ 0.   0.   1.   0.2  0. ] ranking score=0.000


### (3-4) モデルの評価

calc_mean_random または calc_mean の両関数を用いて行うとのことです。

（2017/05/03現在、詳細については不明）

In [13]:
print('Random ranking:', metric.calc_mean_random(Eqids, Ey))
print('Our model:', metric.calc_mean(Eqids, Ey, Epred))

Random ranking: 0.717637387847
Our model: 0.422676641517
